In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt

In [3]:
database = sql.connect("database.sqlite")
tables = pd.read_sql("Select * FROM sqlite_master WHERE type = 'table';", con=database)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [4]:
Teams = pd.read_sql("SELECT * FROM Team;", database)
teams = pd.read_sql("""SELECT team_api_id, team_long_name FROM Team;""", database)
Match = pd.read_sql("SELECT * FROM Match;", database)
match = pd.read_sql("""SELECT Match.country_id, Match.season, Match.home_team_api_id, Match.away_team_api_id, Match.home_team_goal, Match.away_team_goal
                        FROM Match;""", database)
Country = pd.read_sql("SELECT * from Country;", database)
Leagues = pd.read_sql("SELECT * FROM League;", database)
leagues = pd.read_sql("""SELECT Country.id as Country_id, Country.name as Country_name, League.name as League_name
                        FROM League
                        JOIN Country ON Country.id = League.country_id;""", database)

In [5]:
match = pd.merge(match, teams, left_on='home_team_api_id', right_on='team_api_id')
match = pd.merge(match, teams, left_on='away_team_api_id', right_on='team_api_id')
match = pd.merge(match, leagues, left_on='country_id', right_on='Country_id')

In [6]:
def goal_to_score(a):
    if a > 0:
        return 3
    elif a == 0:
        return 1
    else:
        return 0

In [7]:
data = match.iloc[:, [1, 4, 5, 7, 9, 12]]
data['home_team_score'] = (data['home_team_goal']-data['away_team_goal']).map(goal_to_score)
data['away_team_score'] = (data['away_team_goal']-data['home_team_goal']).map(goal_to_score)
data['W'] = data['away_team_goal'] < data['home_team_goal']
data['D'] = data['away_team_goal'] == data['home_team_goal']
data['L'] = data['away_team_goal'] > data['home_team_goal']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [8]:
data.isnull().any()

season              False
home_team_goal      False
away_team_goal      False
team_long_name_x    False
team_long_name_y    False
League_name         False
home_team_score     False
away_team_score     False
W                   False
D                   False
L                   False
dtype: bool

No missing data

In [9]:
seasons = data['season'].unique().tolist()
leagues_list = leagues['League_name'].unique().tolist()
team_list = teams['team_long_name'].unique().tolist()

In [10]:
group_1 = data.groupby(["season", "League_name", "team_long_name_x"]).agg('sum')
group_2 = data.groupby(["season", "League_name", "team_long_name_y"]).agg('sum')
group_list = list(group_2)
group_list[0], group_list[1], group_list[2], group_list[3], group_list[4], group_list[5], group_list[6] = group_list[1], group_list[0], group_list[3], group_list[2], group_list[6], group_list[5], group_list[4]
group_2.columns = group_list
Team_data_all = (group_1 + group_2).iloc[:, [2, 0, 1, 5, 3, 6]].astype(int)
Team_data_all.columns = ['W','D','L','GF','GA','Pts']
Team_data_all['GD'] = Team_data_all['GF'] - Team_data_all['GA']
Team_data_all

W   D   L  GF  \
season    League_name              team_long_name_x                          
2008/2009 Belgium Jupiler League   Beerschot AC             11   9  14  44   
                                   Club Brugge KV           18   5  11  59   
                                   FCV Dender EH             9   8  17  44   
                                   KAA Gent                 17   8   9  67   
                                   KRC Genk                 15   5  14  48   
...                                                         ..  ..  ..  ..   
2015/2016 Switzerland Super League FC Thun                  10  11  15  45   
                                   FC Vaduz                  7  15  14  44   
                                   FC Zürich                 7  13  16  48   
                                   Grasshopper Club Zürich  15   8  13  65   
                                   Lugano                    9   8  19  46   

                                                            GA  Pts  GD  
season    League_name              team_long_name_x                      
2008/2009 Belgium Jupiler League   Beerschot AC             42   42   2  
                                   Club Brugge KV           50   59   9  
                                   FCV Dender EH            58   35 -14  
                                   KAA Gent                 42   59  25  
                                   KRC Genk                 51   50  -3  
...                                                         ..  ...  ..  
2015/2016 Switzerland Super League FC Thun                  54   41  -9  
                                   FC Vaduz                 60   36 -16  
                                   FC Zürich                71   34 -23  
                                   Grasshopper Club Zürich  56   53   9  
                                   Lugano                   75   35 -29  

[1478 rows x 7 columns]

Numbers of participating teams of different countries along time.

In [11]:
participating = data.groupby(["season", "League_name"]).agg('nunique')["team_long_name_x"]
participating

season     League_name             
2008/2009  Belgium Jupiler League      18
           England Premier League      20
           France Ligue 1              20
           Germany 1. Bundesliga       18
           Italy Serie A               20
                                       ..
2015/2016  Poland Ekstraklasa          16
           Portugal Liga ZON Sagres    18
           Scotland Premier League     12
           Spain LIGA BBVA             20
           Switzerland Super League    10
Name: team_long_name_x, Length: 88, dtype: int64

View the league ranking of a year

In [12]:
Team_data_all.loc[(seasons[0], leagues_list[9])].sort_values(by='Pts', ascending=False, na_position='first')

,W,D,L,GF,GA,Pts,GD
team_long_name_x,,,,,,,
FC Barcelona,27,6,5,105,35,87,70
Real Madrid CF,25,3,10,83,52,78,31
Sevilla FC,21,7,10,54,39,70,15
Atlético Madrid,20,7,11,80,57,67,23
Villarreal CF,18,11,9,61,54,65,7
Valencia CF,18,8,12,68,54,62,14
RC Deportivo de La Coruña,16,10,12,48,47,58,1
Málaga CF,15,10,13,55,59,55,-4
RCD Mallorca,14,9,15,53,60,51,-7


View the historical Pts of a team

In [13]:
Team_data_all.xs('Liverpool', level='team_long_name_x')

,,W,D,L,GF,GA,Pts,GD
season,League_name,,,,,,,
2008/2009,England Premier League,25,11,2,77,27,86,50
2009/2010,England Premier League,18,9,11,61,35,63,26
2010/2011,England Premier League,17,7,14,59,44,58,15
2011/2012,England Premier League,14,10,14,47,40,52,7
2012/2013,England Premier League,16,13,9,71,43,61,28
2013/2014,England Premier League,26,6,6,101,50,84,51
2014/2015,England Premier League,18,8,12,52,48,62,4
2015/2016,England Premier League,16,12,10,63,50,60,13


Which team is the one with the most goals in a single season?

In [17]:
Team_data_all.sort_values(by='GF', ascending=False, na_position='first').head()

,,,W,D,L,GF,GA,Pts,GD
season,League_name,team_long_name_x,,,,,,,
2011/2012,Spain LIGA BBVA,Real Madrid CF,32,4,2,121,32,100,89
2014/2015,Spain LIGA BBVA,Real Madrid CF,30,2,6,118,38,92,80
2012/2013,Spain LIGA BBVA,FC Barcelona,32,4,2,115,40,100,75
2011/2012,Spain LIGA BBVA,FC Barcelona,28,7,3,114,29,91,85
2015/2016,Spain LIGA BBVA,FC Barcelona,29,4,5,112,29,91,83


2011-2012 Real Madrid 121 goals.